# 🏎️ MIR GPU NITRO-BATCH: High-Performance Audio Analyzer
### *"Breaking the I/O Barrier with Multi-Threading & CUDA"*

Este cuaderno es la versión definitiva de alto rendimiento. Para eliminar el cuello de botella que viste (63s por lote), hemos implementado:
1. **Parallel I/O (CPU Multi-threading)**: Cargamos múltiples archivos del disco al mismo tiempo.
2. **Vectorized GPU Processing**: Procesamos el bloque de audio en la GPU en milisegundos.
3. **Fast-Plotting**: Optimizamos el renderizado de gráficos para que no detenga el motor.

---

In [ ]:
!pip install -q nnAudio torchaudio torch pandas matplotlib tqdm

import os
import torch
import torchaudio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import shutil
import time
import warnings
from concurrent.futures import ThreadPoolExecutor
from nnAudio import features
from IPython.display import Audio, display, FileLink
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")
import librosa
import librosa.display

# Detección de Hardware
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 MODO NITRO ACTIVADO: {device.upper()}")

plt.style.use('dark_background')
plt.rcParams.update({'font.size': 14, 'figure.figsize': (16, 8)})

OUTPUT_DIR = 'nitro_gpu_results'
os.makedirs(os.path.join(OUTPUT_DIR, 'plots'), exist_ok=True)

In [ ]:
SAMPLE_RATE = 44100
BATCH_SIZE = 12 # Ajustado para potencia y seguridad de memoria
SAVE_PLOTS = True # ¡Ponlo en False para velocidad sónica pura!

# Inicializar Motores Espectrales en GPU
mel_gpu = features.MelSpectrogram(sr=SAMPLE_RATE, n_fft=2048, n_mels=128).to(device)
cqt_gpu = features.CQT2010v2(sr=SAMPLE_RATE, hop_length=512, fmin=32.7, n_bins=84).to(device)

print(f"✨ Kernels listos. Lotes de {BATCH_SIZE} pistas cargados en paralelo.")

## 🧬 2. Motor de Carga y Análisis Paralelo

In [ ]:
def parallel_loader(paths):
    """Carga archivos de audio en paralelo usando hilos de CPU."""
    def load_one(p):
        wav, sr = torchaudio.load(p)
        if sr != SAMPLE_RATE:
            wav = torchaudio.transforms.Resample(sr, SAMPLE_RATE)(wav)
        return torch.mean(wav, dim=0) # Convertir a mono

    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        return list(executor.map(load_one, paths))

def process_nitro_batch(file_paths):
    # 1. CARGA MULTI-THREADED (Adiós cuello de botella I/O)
    waveforms = parallel_loader(file_paths)
    metadata = [{"name": os.path.basename(p), "len": w.shape[0]} for p, w in zip(file_paths, waveforms)]
    
    # 2. PADDING Y BATCHING
    max_len = max(w.shape[0] for w in waveforms)
    batch_wav = torch.zeros(len(waveforms), max_len).to(device)
    for i, wav in enumerate(waveforms):
        batch_wav[i, :wav.shape[0]] = wav

    # 3. PROCESAMIENTO GPU (Milisegundos)
    t_start = time.time()
    with torch.no_grad():
        mel_db = torchaudio.transforms.AmplitudeToDB()(mel_gpu(batch_wav)).cpu().numpy()
        cqt_db = torchaudio.transforms.AmplitudeToDB()(torch.abs(cqt_gpu(batch_wav))).cpu().numpy()
    gpu_time = time.time() - t_start

    # 4. EXTRACCIÓN DE DATOS Y PLOTTING (Post-proceso)
    batch_data = []
    for i in range(len(file_paths)):
        fname = metadata[i]['name']
        
        if SAVE_PLOTS:
            plt.figure(figsize=(12, 8))
            plt.subplot(2,1,1)
            librosa.display.specshow(mel_db[i], sr=SAMPLE_RATE, x_axis='time', y_axis='mel', cmap='magma')
            plt.title(f"{fname} | GPU: {gpu_time/len(file_paths):.4f}s/it")
            plt.subplot(2,1,2)
            librosa.display.specshow(cqt_db[i], sr=SAMPLE_RATE, x_axis='time', y_axis='cqt_note', cmap='inferno')
            plt.tight_layout()
            plt.savefig(os.path.join(OUTPUT_DIR, 'plots', f"{fname}_nitro.png"), dpi=80)
            plt.close()
        
        batch_data.append({"file": fname, "gpu_speed": gpu_time/len(file_paths)})
        
    return batch_data

## 🚀 3. Ejecución Nitro

In [ ]:
DATASET_PATH = '/kaggle/input/datasets/danieldobles/slavic-songs'
if not os.path.exists(DATASET_PATH): DATASET_PATH = 'Slavic Data_Set'
all_files = [os.path.join(DATASET_PATH, f) for f in os.listdir(DATASET_PATH) if f.endswith(('.mp3', '.wav', '.flac')) and f != 'REF.flac' ]

all_results = []
print(f"🔥 NITRO MODE ON: Procesando {len(all_files)} archivos en lotes de {BATCH_SIZE}")

total_start = time.time()
for i in tqdm(range(0, len(all_files), BATCH_SIZE), desc="Shredding Audio"):
    batch_paths = all_files[i : i + BATCH_SIZE]
    try:
        batch_info = process_nitro_batch(batch_paths)
        all_results.extend(batch_info)
    except Exception as e:
        print(f"⚠️ Error en lote {i}: {e}")

total_time = time.time() - total_start
print(f"\n🏆 CARNICERÍA COMPLETADA EN {total_time:.2f}s")
print(f"⚡ Velocidad Promedio Real: {total_time/len(all_files):.3f}s por pista (incluyendo I/O y Gráficos)")

shutil.make_archive('MIR_NITRO_RESULTS', 'zip', OUTPUT_DIR)
display(FileLink('MIR_NITRO_RESULTS.zip', result_html_prefix="🚀 DESCARGA EL RESULTADO NITRO: "))